<a href="https://colab.research.google.com/github/Soob00/20251R0136COSE47101/blob/dataset2_employed/dataset2/notebooks/dataset2_employed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd

df = pd.read_csv("/content/20%이하인 결측치 삭제된 데이터.csv")
print(df.head())

print(df.columns.tolist())
print(df.isnull().sum())
print(df.describe(include='all'))

   YEAR       AGE  EDUC  ETHNIC  RACE  GENDER  SPHSERVICE  CMPSERVICE  \
0  2022  1.000000 -9.00       3     3       0           0           1   
1  2022  0.538462  0.50       3    -9       0           0           1   
2  2022  0.230769  0.25       3     4       1           0           1   
3  2022  0.153846  0.75       1     6       0           0           1   
4  2022  0.923077  0.75       4     5       0           0           1   

   OPISERVICE  RTCSERVICE  ...  ODDFLG  PDDFLG  PERSONFLG  SCHIZOFLG  \
0           0           0  ...       0       0          0          1   
1           1           0  ...       0       0          0          0   
2           0           0  ...       0       0          0          0   
3           0           0  ...       0       0          0          0   
4           0           0  ...       0       0          0          0   

   ALCSUBFLG  OTHERDISFLG  STATEFIP  DIVISION  REGION       CASEID  
0          0            0         1         6       3  2022

In [25]:
missing_cols = df.columns[df.isnull().any()]
df[missing_cols].isnull().sum()

missing_rows = df[df.isnull().any(axis=1)]
print(missing_rows)

Empty DataFrame
Columns: [YEAR, AGE, EDUC, ETHNIC, RACE, GENDER, SPHSERVICE, CMPSERVICE, OPISERVICE, RTCSERVICE, IJSSERVICE, MH1, MH2, MH3, SUB, MARSTAT, SMISED, SAP, EMPLOY, DETNLF, VETERAN, LIVARAG, NUMMHS, TRAUSTREFLG, ANXIETYFLG, ADHDFLG, CONDUCTFLG, DELIRDEMFLG, BIPOLARFLG, DEPRESSFLG, ODDFLG, PDDFLG, PERSONFLG, SCHIZOFLG, ALCSUBFLG, OTHERDISFLG, STATEFIP, DIVISION, REGION, CASEID]
Index: []

[0 rows x 40 columns]


# Step1. 기초통계비교

# 1. **t-test : 고용상태에 따라 인구사회학적/임상적 특징이 다른가?**

In [38]:
from scipy.stats import ttest_ind

# (0, 0.25 = 고용 / 0.75, 1 = 비고용) 기준으로 binary 생성 (1 = 고용, 0 = 비고용)
employed_filter = df[df['EMPLOY'].isin([0, 0.25, 0.75, 1])].copy()

employed_filter['employed_status'] = employed_filter['EMPLOY'].apply(lambda x: 1 if x in [0, 0.25] else 0)

# 수치형 변수: AGE, NUMMHS 등 평균 차이 분석
numeric_cols = ['AGE', 'NUMMHS']
for col in numeric_cols:
    employed = employed_filter[employed_filter['employed_status'] == 1][col]
    unemployed = employed_filter[employed_filter['employed_status'] == 0][col]
    t_stat, p = ttest_ind(employed, unemployed, equal_var=False)
    print(f"[T-test] {col}: p = {p:.4f}")

# 범주형 변수: CHI-SQUARE
categorical_cols = [
    'EDUC', 'GENDER', 'ETHNIC', 'RACE', 'MARSTAT',
    'TRAUSTREFLG', 'ANXIETYFLG', 'ADHDFLG', 'CONDUCTFLG',
    'DELIRDEMFLG', 'BIPOLARFLG', 'DEPRESSFLG', 'ODDFLG', 'PDDFLG',
    'PERSONFLG', 'SCHIZOFLG', 'ALCSUBFLG', 'OTHERDISFLG', 'SAP', 'VETERAN'
]
from scipy.stats import chi2_contingency

for col in categorical_cols:
    ct = pd.crosstab(employed_filter['employed_status'], employed_filter[col])
    if ct.shape[0] == 2 and ct.shape[1] > 1:  # 조건 만족 시만 계산
        chi2, p_val, _, _ = chi2_contingency(ct)
        print(f"{col}: p = {p_val:.4f}")

[T-test] AGE: p = 0.0000
[T-test] NUMMHS: p = 0.0000
EDUC: p = 0.0000
GENDER: p = 0.0000
ETHNIC: p = 0.0000
RACE: p = 0.0000
MARSTAT: p = 0.0000
TRAUSTREFLG: p = 0.0000
ANXIETYFLG: p = 0.0000
ADHDFLG: p = 0.9013
CONDUCTFLG: p = 0.0000
DELIRDEMFLG: p = 0.0000
BIPOLARFLG: p = 0.8930
DEPRESSFLG: p = 0.0000
ODDFLG: p = 0.0000
PDDFLG: p = 0.0000
PERSONFLG: p = 0.0003
SCHIZOFLG: p = 0.0000
ALCSUBFLG: p = 0.0213
OTHERDISFLG: p = 0.0000
SAP: p = 0.0000
VETERAN: p = 0.0000


### 결과 설명
p-value가 0.05보다 작은 경우, 고용상태와 유의미한 관계로 판단할 수 있다.

- AGE(나이), NUMMHS(정신건강 서비스 이용 횟수), EDUC(교육 수준), GENDER(성별), ETHNIC(민족), RACE(인종), MARSTAT(혼인 상태), TRAUSTREFLG(법원 명령 여부), ANXIETYFLG(불안장애 여부), CONDUCTFLG(품행장애 여부), DELIRDEMFLG(섬망/치매 여부), DEPRESSFLG(우울증 여부), ODDFLG(반항성 장애 여부), PDDFLG(발달장애 여부), PERSONFLG(인격장애 여부), SCHIZOFLG(정신분열증 여부), ALCSUBFLG(알코올 관련 문제 여부), OTHERDISFLG(기타 질환 여부), SAP(약물치료 프로그램 참여 여부), VETERAN(재향군인 여부)

- ADHDFLG(ADHD 여부)와 BIPOLARFLG(양극성 장애 여부)는 고용여부와 큰 연관이 없다.

# 2. 입원 여부에 따라 고용 가능성이 달라지는가?

In [42]:
import pandas as pd
from scipy.stats import chi2_contingency

# EMPLOY: 0, 0.25 : '고용', 0.75, 1 : '비고용'
df['employed_status'] = df['EMPLOY'].apply(lambda x: 1 if x in [0, 0.25] else 0)

ct = pd.crosstab(df['SPHSERVICE'], df['employed_status'])

chi2, p_val, dof, expected = chi2_contingency(ct) # 카이제곱 실행

print(f"p = {p_val:.4f}")


p = 0.0000


### 결과 설명
입원 여부와 고용 여부는 서로 독립인가?
>> **p-value 값이 작기 때문에** 입원 여부와 고용 여부가 서로 독립이 아닌, 유의미한 관계가 있다고 판단 가능.

**따라서, 회복, 즉 고용에 있어, 입원은 중요한 연과관계가 있다고 볼 수 있다.**